In [1]:
import os
import pyspark
conf = pyspark.SparkConf()

conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf)

spark = pyspark.SQLContext.getOrCreate(sc)


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/15 19:22:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/opt/conda/envs/bigdata/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
#Question 1 Code
import pyspark.sql.functions as func
Bakery = spark.read.option("inferSchema", "true").option("header","true").csv("/home/jovyan/shared/hw2_data/Bakery.csv")
Bakery_split = Bakery.withColumn("date", func.to_date(func.col("DateTime"))).withColumn("hour", func.hour(func.col("DateTime")))
Bakery_split.groupBy("Items","date","hour").count()

Items,date,hour,count
Bread,2016-10-31,10,2
Medialuna,2016-04-11,8,1
Granola,2016-05-11,8,1
Farm House,2016-06-11,11,1
Tartine,2016-08-11,14,1
Truffles,2016-12-11,11,2
Bread,2016-12-11,16,2
Muffin,2016-11-15,16,1
Pastry,2016-11-18,10,2
Coffee,2016-11-28,8,1


In [3]:
#Question 2 Code

from pyspark.sql.types import StructField, StructType, StringType, LongType
from pyspark.sql import Row
x = Bakery.groupBy("Daypart","Items", "DayType").count().sort("Daypart","DayType",func.desc("count"))
Daypart = Bakery.select("Daypart").distinct().rdd.map(lambda r: r[0]).collect()
DayType = Bakery.select("DayType").distinct().rdd.map(lambda r: r[0]).collect()

MyQ2Schema = StructType([StructField("Daypart", StringType(), True),
                        StructField("food", StringType(), True),
                        StructField("DayType", StringType(), True)])

length = len(Daypart) * len(DayType)
test = x.filter((func.col("DayType") == DayType[0]) & (func.col("Daypart") == Daypart[0])).drop('count').take(3)
QuestionTwoDataFrame = spark.createDataFrame([test[0]],MyQ2Schema)
newRow = spark.createDataFrame([test[1]])
QuestionTwoDataFrame =  QuestionTwoDataFrame.union(newRow)
newRow = spark.createDataFrame([test[2]])
QuestionTwoDataFrame =  QuestionTwoDataFrame.union(newRow)

for i in range(1,len(Daypart)):
    test = x.filter((func.col("DayType") == DayType[0]) & (func.col("Daypart") == Daypart[i])).drop('count').take(3)
    for j in range(3):
        newRow = spark.createDataFrame([test[j]])
        QuestionTwoDataFrame =  QuestionTwoDataFrame.union(newRow)    

for i in range(len(Daypart)):
    test = x.filter((func.col("DayType") == DayType[1]) & (func.col("Daypart") == Daypart[i])).drop('count').take(3)
    for j in range(3):
        newRow = spark.createDataFrame([test[j]])
        QuestionTwoDataFrame =  QuestionTwoDataFrame.union(newRow)    

FinalQuestionTwo = QuestionTwoDataFrame.groupBy('Daypart', 'DayType').agg(func.collect_set('food').alias('words')).show()

+---------+-------+--------------------+
|  Daypart|DayType|               words|
+---------+-------+--------------------+
|  Evening|Weekday|[Coffee, Bread, Tea]|
|  Morning|Weekday|[Coffee, Pastry, ...|
|Afternoon|Weekday|[Coffee, Bread, Tea]|
|    Night|Weekday|[Valentine's card...|
|  Evening|Weekend|[Coffee, Tshirt, ...|
|  Morning|Weekend|[Coffee, Pastry, ...|
|Afternoon|Weekend|[Coffee, Bread, Tea]|
|    Night|Weekend|[Scandinavian, Ho...|
+---------+-------+--------------------+



In [4]:
#Question 3 Code
Resturants = spark.read.option("inferSchema", "true").option("header","true").option("delimiter",";").csv("/home/jovyan/shared/hw2_data/Restaurants_in_Durham_County_NC.csv")
Resturants.groupBy("Rpt_Area_Desc").count()

Rpt_Area_Desc,count
Bed&Breakfast Home,4
Summer Camps,4
Institutions,30
Local Confinement,2
Mobile Food,147
School Buildings,89
null,13
Summer Food,242
Swimming Pools,420
Day Care,173


In [5]:
#Question 4 Code

from pyspark.sql.types import FloatType
Time_Data = spark.read.option("inferSchema", "true").option("header","true").csv("/home/jovyan/shared/hw2_data/populationbycountry19802010millions.csv")
names = Time_Data.columns

names = Time_Data.columns
for i in range(1,len(names) - 1):
    Time_Data = Time_Data.withColumn(names[i + 1] + " subtract " + names[i], ((func.col(names[i + 1]) - func.col(names[i]))/func.col(names[i])))

colnames = Time_Data.columns

top_percent = Time_Data.groupby().max('1981 subtract 1980').first().asDict()['max(1981 subtract 1980)']
country = Time_Data.where(func.col('1981 subtract 1980') == top_percent).select(func.col('_c0')).rdd.map(lambda r: r[0]).collect()

MyQ4Schema = StructType([StructField("Year", StringType(), True),
                        StructField("Country", StringType(), True),
                        StructField("Percentage", FloatType(), True)])

firstRow = Row(colnames[2],country,top_percent)
QuestionFourDataFrame = spark.createDataFrame([firstRow],MyQ4Schema)


for i in range(29):
    year = colnames[i+3]
    subtraction = colnames[i + 33]
    top_percent = Time_Data.groupby().max(subtraction).first().asDict()['max('+subtraction+')']
    country = Time_Data.where(func.col(subtraction) == top_percent).select(func.col(colnames[0])).rdd.map(lambda r: r[0]).collect()
    new_row = Row(year,country,top_percent)
    new_row_df = spark.createDataFrame([new_row],MyQ4Schema)
    QuestionFourDataFrame = QuestionFourDataFrame.union(new_row_df)

QuestionFourDataFrame.show()
    


22/03/15 19:24:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1980, 1981
 Schema: _c0, 1980, 1981
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2_data/populationbycountry19802010millions.csv
22/03/15 19:24:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1981, 1982
 Schema: _c0, 1981, 1982
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2_data/populationbycountry19802010millions.csv
22/03/15 19:24:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1982, 1983
 Schema: _c0, 1982, 1983
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2_data/populationbycountry19802010millions.csv
22/03/15 19:24:04 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1983, 1984
 Schema: _c0, 1983, 1984
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2_data/populationbycountry19802010millions.csv
22/03/15 19:24:04 WARN CSVHe

+----+--------------------+-----------+
|Year|             Country| Percentage|
+----+--------------------+-----------+
|1981|    [Western Sahara]|0.121331826|
|1982|    [Western Sahara]|0.111151055|
|1983|     [French Guiana]| 0.14285715|
|1984|             [Qatar]|0.109640576|
|1985|     [French Guiana]|      0.125|
|1986|             [Qatar]|0.087717324|
|1987|     [French Guiana]| 0.11111111|
|1988|    [Cayman Islands]| 0.11010421|
|1989|[United Arab Emir...| 0.06119858|
|1990|          [Djibouti]| 0.12824048|
|1991|            [Jordan]|  0.1127394|
|1992|            [Kuwait]| 0.48633438|
|1993|       [Afghanistan]| 0.13224594|
|1994|       [Afghanistan]|0.087276615|
|1995|           [Burundi]| 0.07222489|
|1996|            [Rwanda]| 0.19614178|
|1997|[Falkland Islands...|      0.215|
|1998|           [Liberia]|  0.1201745|
|1999|[Falkland Islands...| 0.07692308|
|2000|        [Montserrat]| 0.16863905|
+----+--------------------+-----------+
only showing top 20 rows



In [6]:
#Question 5 Code

from pyspark.sql.functions import regexp_replace
import re
Text_file = spark.read.option("inferSchema", "true").option("lineSep",".").text("/home/jovyan/shared/hw2_data/internet_archive_scifi_v3.txt")
Text_file=Text_file.withColumn("value", func.lower(func.col("value")))
Text_file = Text_file.withColumn("value",regexp_replace("value", "[^0-9a-zA-Z$]+", " "))
Text_file = Text_file.withColumn('word', func.explode(func.split(func.col('value'), '\s+'))).groupBy('word')\
    .count()\
    .show()

+------------+-----+
|        word|count|
+------------+-----+
|        some|42889|
|        hope| 5880|
|       still|28742|
|  lieutenant| 1370|
|         fog|  925|
|    tortured|  288|
|       trail| 1209|
|       those|18840|
|       inner| 2029|
|    affixing|    2|
|         few|18027|
|     reddish|  224|
|        guts|  509|
|   arguments|  315|
|   connected|  723|
|accumulation|   98|
|      harder|  884|
|   traveling|  746|
|     skidded|  186|
|   amplifier|  123|
+------------+-----+
only showing top 20 rows



In [17]:
Text_file2 = spark.read.option("inferSchema", "true").option("lineSep",".").text("/home/jovyan/shared/hw2_data/internet_archive_scifi_v3.txt")
Text_file2 =Text_file2.withColumn("value", func.lower(func.col("value")))
Text_file2 = Text_file2.withColumn("value",regexp_replace("value", "[^0-9a-zA-Z$]+", " "))
Text_file2 = Text_file2.na.drop()
bigrams = Text_file2.rdd.flatMap(lambda xs: (tuple(x) for x in zip(xs, xs[1:]))).map(lambda x: (x))

#bigrams.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
test = bigrams.toDF()

ValueError: RDD is empty

In [10]:
r = bigrams.collect()
[f(e) for e in r]

[]